In [1]:

import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, ConcatDataset
from torch.utils.data import Dataset
import torch.optim as optim
from torchvision import transforms
import numpy as np
import torch
from torch import nn
import os
from torch.utils.data import DataLoader, random_split
from torch.nn import DataParallel
from torch.utils.data import Subset
import shutil
from codes.helpers import FocalLoss
import cv2

In [2]:
generator_name_to_load = "final_model.pth"

batch_size = 1
device = "cuda"
logdir = "logs/"

In [3]:
from torchvision import transforms
from torch.utils.data import Dataset
from PIL import Image
import os
import numpy as np


class Fashion_Data(Dataset):
    def __init__(self, folder_train, augmentation=None):
        self.folder_train = folder_train
        self.transform = transforms.Compose([transforms.ToTensor()])
        
        self.filenames = [f for f in os.listdir(folder_train) if os.path.isfile(os.path.join(folder_train, f))]

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_name_train = os.path.join(self.folder_train, self.filenames[idx])

        img_train = Image.open(img_name_train).convert('L')
        img_train = self.transform(img_train)

        return img_train


In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

test_folder = "logs/input/"
test_set = Fashion_Data(test_folder)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=False)



In [5]:
# The diagram is provided to understand the architecture
from architecture.segnet import SegNet

generator = SegNet(in_channels=1, out_channels=1).to(device)
generator = DataParallel(generator)
generator.load_state_dict(torch.load(f'logs/{generator_name_to_load}'))

<All keys matched successfully>

In [6]:
# The code to make sure the model input and output are correct, we can test the first batch to fed into the network
data = next(iter(test_dataloader))

test_data = data
output_test = generator(test_data.to(device))
print(output_test.shape)

torch.Size([1, 1, 1024, 768])


In [7]:
from tqdm import tqdm
from torchvision.utils import save_image

# generator.eval()
pbar = tqdm(test_dataloader)
for idx, x in enumerate(pbar):
    x = x.to(device, dtype=torch.float32)
    output = generator(x)
    for j, gen_image in enumerate(output):
        save_image(gen_image, os.path.join("logs", "output", f"generated_{idx}_{j}.png"))

100%|██████████| 12/12 [00:01<00:00,  6.72it/s]
